In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.utils.data as Data

seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

epochs= 100
learning_rate = 1e-4
from PIL import  Image,ImageDraw
import numpy as np


data1 = []
data2= []

for i in range(20000):
    img = Image.new('RGB', (32, 32), color = 'white')
    row,col,ch= np.shape(img)
    mean = 0
    var = 1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    noisy = img + gauss
    im2 = Image.fromarray(noisy,'RGB')
    r = np.random.randint(1,101)
    draw = ImageDraw.Draw(im2)
    if r%2==0:
        draw.rectangle([9,12,12,15],fill='red',outline='red')
        data1.append(np.asarray(im2))
    else:
        draw.rectangle([21, 12, 24, 15], fill='red', outline='red')
        data1.append(np.asarray(im2))



for i in range(20000):
    img = Image.new('RGB', (32, 32), color = 'white')
    row,col,ch= np.shape(img)
    mean = 0
    var = 1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    noisy = img + gauss
    im2 = Image.fromarray(noisy,'RGB')
    r = np.random.randint(1,101)
    draw = ImageDraw.Draw(im2)
    if r%2==0:
        draw.rectangle([21, 12, 24, 15], fill='red', outline='red')
        data2.append(np.asarray(im2))
    else:
        draw.rectangle([9,12,12,15],fill='red',outline='red')
        data2.append(np.asarray(im2))


y0 = np.array(data1, dtype=np.float32)
y1 = np.array(data2, dtype=np.float32)
y0 = np.transpose(y0, (0, 3, 2, 1))# This to make channels first for CNN
y1 = np.transpose(y1, (0, 3, 2, 1)) 


x = torch.from_numpy(y0)
y = torch.from_numpy(y1)



torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=250,

    shuffle=True,

    num_workers=0,

)


class CNN(nn.Module):
    """CNN."""

    def __init__(self):
        """CNN Builder."""
        super(CNN, self).__init__()

        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )


        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )


    def forward(self, x):
        """Perform forward."""
        
        # conv layers
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        x = self.fc_layer(x)

        return x

#  use gpu if available

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
model = CNN()






# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()
# mean-squared error loss
def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)

    covf = torch.matmul((fx.t()), fx) / Nsamples

    covg = torch.matmul((gy.t()), (gy)) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h

for epoch in range(epochs):
    loss = 0
    for x,y in loader:
        # reshape mini-batch data to [N, 784] matrix
        # load it to the active device
        #print(batch_features[0].shape)
        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()
        # compute reconstructions        
        # compute training reconstruction loss
        train_loss = h_score(model(x),model(y))
        #train_loss = criterion(model(x),model(y))
        # compute accumulated gradients
        train_loss.backward()
        
        # perform parameter update based on current gradients
        optimizer.step()
        
        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()
    
    # compute the epoch training loss
    loss = loss / len(loader)
    
    # display the epoch training loss
    print("epoch : {}/{}, recon loss = {:.8f}".format(epoch + 1, epochs,loss))
    #print(np.shape(model.encoder_hidden_layer.weight.data),np.shape(model_dec.decoder_output_layer.weight.data))




epoch : 1/100, recon loss = -0.00001037
epoch : 2/100, recon loss = 0.00037114
epoch : 3/100, recon loss = -0.00048991
epoch : 4/100, recon loss = -0.00513173
epoch : 5/100, recon loss = -0.06180491
epoch : 6/100, recon loss = -0.30113275
epoch : 7/100, recon loss = -0.60111449
epoch : 8/100, recon loss = -0.92566966
epoch : 9/100, recon loss = -1.08656090
epoch : 10/100, recon loss = -1.43537042
epoch : 11/100, recon loss = -1.58237745
epoch : 12/100, recon loss = -1.80057842
epoch : 13/100, recon loss = -1.86531052
epoch : 14/100, recon loss = -1.93677470
epoch : 15/100, recon loss = -2.11085800
epoch : 16/100, recon loss = -2.25805878
epoch : 17/100, recon loss = -2.40782664
epoch : 18/100, recon loss = -2.59677409
epoch : 19/100, recon loss = -2.63035960
epoch : 20/100, recon loss = -2.75634891
epoch : 21/100, recon loss = -2.85311681
epoch : 22/100, recon loss = -2.92586933
epoch : 23/100, recon loss = -2.94641345
epoch : 24/100, recon loss = -3.06577549
epoch : 25/100, recon loss

In [5]:
blank = cv2.imread('blank_32x32.png')
b = blank.astype('float32')
b = torch.from_numpy(b)

for i in range(5,15):
    print("x: ", model(x[i].reshape(1,3,32,32)))
    print("y:", model(y[i].reshape(1,3,32,32)))

print("b:", model(b.reshape(1,3,32,32)))

x:  tensor([[ 0.2564, -0.9170, -0.4204, -0.9176, -0.5451,  1.0586, -0.4545, -1.2155,
          0.1261, -0.7568]], grad_fn=<AddmmBackward>)
y: tensor([[ 0.2618, -0.8505, -0.4062, -0.9135, -0.5245,  1.0455, -0.4824, -1.1826,
          0.2019, -0.7294]], grad_fn=<AddmmBackward>)
x:  tensor([[ 0.2006, -0.7440, -0.3341, -0.9038, -0.4963,  1.0493, -0.5965, -1.0724,
          0.1351, -0.6476]], grad_fn=<AddmmBackward>)
y: tensor([[ 0.2653, -0.9275, -0.3849, -0.9667, -0.5870,  1.1170, -0.5546, -1.2390,
          0.2306, -0.7902]], grad_fn=<AddmmBackward>)
x:  tensor([[ 0.3067, -1.0308, -0.4228, -1.0822, -0.6488,  1.2524, -0.5256, -1.4404,
          0.2580, -0.8963]], grad_fn=<AddmmBackward>)
y: tensor([[ 0.3277, -0.8375, -0.3105, -0.9817, -0.5107,  1.1314, -0.4314, -1.2997,
          0.2884, -0.8345]], grad_fn=<AddmmBackward>)
x:  tensor([[ 0.2699, -0.9007, -0.4099, -0.9531, -0.6099,  1.1198, -0.5056, -1.2708,
          0.1776, -0.7638]], grad_fn=<AddmmBackward>)
y: tensor([[ 0.2573, -0.8342, 